In [1]:
# Parameters
home = "D:\\Sharpcap\\2020-02-23\\Capture\\smallcaps\\34g-9ms-1200-1600-0\\7"
nsig = 3


In [2]:
#Libraries
import numpy as np
import astropy as ap
import math
import time
from copy import copy
import matplotlib.pyplot as plt
import photutils
from astropy.io import fits
from astropy.stats import sigma_clipped_stats, SigmaClip
from photutils import centroid_com, centroid_1dg, centroid_2dg,find_peaks, StdBackgroundRMS,fit_2dgaussian
from astropy.io.fits import getheader
import scipy.optimize as opt
import os
from astropy import units as u
import matplotlib.colors as colors
from astropy.wcs import WCS
import shutil
from copy import copy
from photutils import detect_threshold
import glob
from photutils import detect_sources
from ccdproc import cosmicray_lacosmic as lacosmic
from natsort import natsorted
import shutil
from photutils import source_properties
import fnmatch
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
import sys
args=sys.argv
import papermill as pm

In [3]:
# os.chdir(r'D:\Sharpcap\2020-02-23\Capture\smallcaps\34g-9ms-1200-1600-0\3\calibrated')
# data=fits.getdata('calibrated503.fits')
# if data.max() > 6000:
#     print(data.argmax())
#     print('Saturation warning')

In [4]:
# maxind = np.unravel_index(np.argmax(data, axis=None), data.shape)
# ind

In [5]:
def jeffwrite(s,e):
            w=1
            os.chdir(light)
            lightfiles=glob.glob('Capture*fits')
            os.chdir(dark)
            darkcube=glob.glob('Capture*fits')
            raw_image_data = {}
            for image_name in darkcube: raw_image_data[image_name] = fits.getdata(image_name)
            darkcube=natsorted(darkcube)
            darkcube = np.stack([raw_image_data[science_frame] for science_frame in darkcube],axis=0)
            darkmed= np.median(darkcube, axis=0)
            darkcube=[]
            arra=np.zeros((darkmed.shape[0],darkmed.shape[1]))
            whynot=copy(arra)
            #arbitrarily defined as it just needs to be sufficiently high to show where a bright pixel was recorded. 
            arb=100000
            for t in range(s,e): 
                try:
                    if t%150==0:
                        print(t)
                    os.chdir(light)             
                    processed=fits.getdata(lightfiles[t])-darkmed
                    hdr=fits.Header()
                    hdr=getheader(lightfiles[t])
                    proc=copy(processed)
                    if t==s:
                        threshold3 = detect_threshold(proc, nsigma=nsig)
                    arra=arra+(proc-np.mean(proc)) 
                    proc[proc<threshold3]=0
                    proc[proc>threshold3]=arb
                    whynot=whynot+proc 
                    f = 'calibrated'+str(t+1)+'.fits'
                    cal=home + '\calibrated'
                    path = os.path.join(cal, f) 
                    if os.path.exists(path)== True:
                        os.remove(path) 
                    os.chdir(cal)
                    
                    data    = processed
                    hduprim = fits.PrimaryHDU(data,header=hdr)
                    hdul    = fits.HDUList([hduprim])
                    hduprim.scale('int32', bzero=0)
                    hdul.writeto('calibrated'+str(t+1)+'.fits', overwrite=True,output_verify='ignore')
#                 #In the case that a file is corrupted  
                except TypeError:
                    print('There was an error with this file:'+str(lightfiles[t])+'....deletion is suggested')
                    continue
            os.chdir(cal)   
            hdr     = fits.Header()
            data = np.int64(arra)
            hduprim = fits.PrimaryHDU(data,header=hdr)
            hdul    = fits.HDUList([hduprim])
            hdul.writeto('stackedmean.fits', overwrite=True,output_verify='ignore') 
            #approximate estimate as to the serious influence a star will be able to impart to neighbouring stars
            #This can be mathmatically assumed later from the sky position and angular speed but 15 pixels
            
            data = np.int64(whynot)
            hdr     = fits.Header()
            hduprim = fits.PrimaryHDU(data,header=hdr)
            hdul    = fits.HDUList([hduprim])
            hdul.writeto('trail.fits', overwrite=True,output_verify='ignore') 
            
            whynot[whynot>10*arb]=0
            data = np.int64(whynot)
            hdr     = fits.Header()
            hduprim = fits.PrimaryHDU(data,header=hdr)
            hdul    = fits.HDUList([hduprim])
            hdul.writeto('trailcap.fits', overwrite=True,output_verify='ignore') 
            return(arra,proc)

# Access the relavant directory/files

In [6]:
#light and dark must be named 'light' and 'dark' for ease of use
os.chdir(home)
light=home
dark=os.path.dirname(home)+r'\dark'
if os.path.exists('calibrated')== False:
     os.mkdir('calibrated')
cal=home + '\calibrated'        

In [7]:
os.chdir(light)
if 'start' not in locals():
    start=0
if 'end' not in locals():
    end=len(glob.glob('Capture*fits'))  
#jefftry(first file,last file assignment)
ar,gath=jeffwrite(start,end)
#jeffwrite(start,end)

0


 [astropy.io.fits.verify]


150
300
450
